In [1]:
import yolort
print(yolort.__version__)
print(yolort.__file__)

0.7.0a0
/home/guest1/LISHUFEI/jupyter_code/yolov5-rt-stack/yolort/__init__.py


In [2]:
import argparse
import platform
import sys
import time
from pathlib import Path

In [3]:
import pandas as pd

In [4]:
FILE = Path().resolve()
ROOT = FILE.parents[0]  # YOLOrt root directory
print(str(ROOT))
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
print(str(ROOT)in sys.path)

/home/guest1/LISHUFEI/jupyter_code/yolov5-rt-stack
True


### 配置环境

In [5]:
import os
import cv2
import numpy

import torch

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # PCI_BUS_ID” # 按照PCI_BUS_ID顺序从0开始排列GPU设备 
os.environ["CUDA_VISIBLE_DEVICES"]="0"  #设置当前使用的GPU设备仅为0号设备  设备名称为'/gpu:0'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cuda = device.type != 'cpu'

### 导入模型和定义的函数

In [6]:
from yolort.models.yolo import YOLO
from yolort.utils import Visualizer, get_image_from_url, read_image_to_tensor, check_dataset
from yolort.v5 import load_yolov5_model, letterbox, non_max_suppression, scale_coords, attempt_download
from yolort.v5.utils.downloads import safe_download
from yolort.v5.utils.dataloaders import *
from yolort.v5.utils.general import colorstr

### 加载数据

In [7]:
data = ROOT / 'data/coco128.yaml'  # dataset.yaml path
data = check_dataset(data)

In [8]:
# print(data)  

In [9]:
# img_source = "https://huggingface.co/spaces/zhiqwang/assets/resolve/main/bus.jpg"
# # img_source = "https://huggingface.co/spaces/zhiqwang/assets/resolve/main/zidane.jpg"
# img_raw = get_image_from_url(img_source)

In [10]:
# print(img_raw)

In [11]:
# image = letterbox(img_raw, new_shape=(img_size, img_size), stride=stride)[0]
# image = read_image_to_tensor(image)
# image = image.to(device)
# image = image[None]
# print(image.size())

In [12]:
img_size = 640
stride = 64
score_thresh = 0.25
batch_size = 32
nms_thresh = 0.45
single_cls=False # treat as single-class dataset
rect = False
workers=8  # max dataloader workers (per RANK in DDP mode)

In [13]:
# print(batch_size)

In [14]:
#images_source = 
#我现在需要做的是不是把images/train2017的图片加载进来？
task = 'val'
task = task if task in ('train', 'val', 'test') else 'val'  # path to train/val/test images
loader,dataset = create_dataloader(data[task],
                               img_size,
                               stride,
                               batch_size,
                               single_cls,
                               rect=rect,
                               workers=workers,
                               prefix=colorstr(f'{task}: '))


val: Scanning '/home/guest1/LISHUFEI/jupyter_code/datasets/coco128/labels/train2017.cache' images and labels... 128 found, 0 missing, 2 empty, 


In [15]:
# print(len(loader)) #loader 的长度是4 128/32
# print(len(dataset))  # 数据集的长度是128 

In [16]:
# # create_dataloader的返回值是什么？
# print(len(loader))  # batch_size num = 4
# ime = dataset[0] # 读取到的图像是一个元组类型的  ime[2]
# print(dataset[0])
# print(ime[2])
# print(type(ime[2]))

### 加载模型权重

In [17]:
# 如果项目路径下没有的话去指定的路径下下载
model_path = 'yolov5s.pt'
checkpoint_path = attempt_download(model_path)

### 加载yolov5模型

In [18]:
# model_yolov5 = load_yolov5_model(checkpoint_path, fuse=True)
# model_yolov5 = model_yolov5.to(device)
# model_yolov5 = model_yolov5.eval()

In [19]:
# print(model_yolov5.names)  # 也可以用 data['names'] 代替

### 加载yolort模型

In [20]:
model_yolort = YOLO.load_from_yolov5(
    checkpoint_path,
    score_thresh=score_thresh,
    nms_thresh=nms_thresh,
    version="r6.0",
)
model_yolort = model_yolort.eval()
model_yolort = model_yolort.to(device)

### batch == 1

In [30]:
for i, im in enumerate(dataset):
    im_path = im[2]
    img_raw = cv2.imread(im_path)  # opencv set the BGR order as the default
    print(img_raw.shape)
    assert img_raw is not None, f"Not Found Image: {im_path}"
    im = letterbox(img_raw, new_shape=(img_size, img_size), stride=stride)[0]
    im = read_image_to_tensor(im)
    print(im.size())
    im = im.to(device)
    im = im[None]
#     print(im.size())
#     with torch.no_grad():
#         yolort_dets = model_yolort(im)
#         scale_coords(im.shape[2:], yolort_dets[0]['boxes'], img_raw.shape[:-1])
#         v = Visualizer(img_raw, data['names'])
#         # Prepare the prediction labels for the Visualizer
#         v.draw_instance_predictions(yolort_dets[0])
#         v.imshow(scale=0.5)
#         print(len(yolort_dets))
#         print(img_raw.shape[:-1])  # 原始图片的size
#         print(im.shape[2:])
#         print(f"Detection results of image: {i}")
#         print((type(yolort_dets[0])))# 检测结果的类型是dict
#         for key in yolort_dets[0]: 
#             print(key) #用这种方式可以查看到字典中的key值有哪些
#             print(yolort_dets[0][key])
#             print(yolort_dets[0][key].size())
#         print(f"Detection boxes with yolort:\n{yolort_dets[0]['boxes']}")
#         print(f"Detection scores with yolort:\n{yolort_dets[0]['scores']}")
#         print(f"Detection labels with yolort:\n{yolort_dets[0]['labels']}")
        

(480, 640, 3)
torch.Size([3, 512, 640])
(426, 640, 3)
torch.Size([3, 448, 640])
(428, 640, 3)
torch.Size([3, 448, 640])
(425, 640, 3)
torch.Size([3, 448, 640])
(640, 481, 3)
torch.Size([3, 640, 512])
(478, 640, 3)
torch.Size([3, 512, 640])
(500, 381, 3)
torch.Size([3, 640, 512])
(488, 640, 3)
torch.Size([3, 512, 640])
(640, 480, 3)
torch.Size([3, 640, 512])
(426, 640, 3)
torch.Size([3, 448, 640])
(640, 427, 3)
torch.Size([3, 640, 448])
(640, 565, 3)
torch.Size([3, 640, 576])
(426, 640, 3)
torch.Size([3, 448, 640])
(375, 500, 3)
torch.Size([3, 512, 640])
(612, 612, 3)
torch.Size([3, 640, 640])
(425, 640, 3)
torch.Size([3, 448, 640])
(640, 512, 3)
torch.Size([3, 640, 512])
(480, 640, 3)
torch.Size([3, 512, 640])
(427, 640, 3)
torch.Size([3, 448, 640])
(427, 640, 3)
torch.Size([3, 448, 640])
(416, 640, 3)
torch.Size([3, 448, 640])
(480, 640, 3)
torch.Size([3, 512, 640])
(640, 416, 3)
torch.Size([3, 640, 448])
(481, 640, 3)
torch.Size([3, 512, 640])
(480, 640, 3)
torch.Size([3, 512, 640])


In [55]:

s =('%20s' + '%11s' * 6) %  ('Class', 'Images', 'Labels', 'P', 'R', 'mAP@.5', 'mAP@.5:.95')
# print(s)
pbar = tqdm(loader, desc=s, bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')  # progress bar
# print(type(pbar))

for batch_i, (im, targets, paths, shapes) in enumerate(pbar):
    print(type(im)) # Tensor 类
    print(im.size())  # N ,C, H, W
    im = np.ascontiguousarray(im)
    print(type(im)) # numpy.ndarray 类
    print(im.shape)  # N ,C, H, W
#     im = im.transpose((0,3,1,2))
    im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR) # cv2只能处理GBR格式图片
    print(im.shape)  # 从这开始和想要的不一样了
    im = letterbox(im, new_shape=(img_size, img_size), stride=stride)[0]  # lettebox需要的图像是np.ndarray类型的
#     im = Image.fromarray(cv2.cvtColor(im,cv2.COLOR_BGR2RGB))
    if cuda:
#         print(type(im)) # 每个 和 paths 中都存储了32张图片的信息，
#         print(len(im))
#         print(len(paths)) 
#         print(type(paths)) # path 是 list 类型的， 长度为32
        im = read_image_to_tensor(im) 
        print(im.size())
        im = im.to(device)
        targets = targets.to(device)
    yolort_dets = model_yolort(im)



               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:   0%|          | 0/4 [00:00<?, ?it/s]                  

<class 'torch.Tensor'>
torch.Size([32, 3, 640, 640])
<class 'numpy.ndarray'>
(32, 3, 640, 640)
(32, 3, 3)
torch.Size([3, 640, 64])


ValueError: expected 4D input (got 3D input)